<a href="https://colab.research.google.com/github/Khushibung05/Ensemble_learning/blob/main/Stacking_regression_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**DATASET 1: LOAN DEFAULT PREDICTION (CLASSIFICATION)**

In [1]:


import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score,classification_report
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import confusion_matrix



In [20]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

# Path to the dataset (update if needed)
path = '/content/drive/My Drive/Colab Notebooks/Loan_prediction/loan.csv'

# Importing the dataset
df= pd.read_csv(path)
print(df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
    Loan_ID Gender Married Dependents     Education Self_Employed  \
0  LP001002   Male      No          0      Graduate            No   
1  LP001003   Male     Yes          1      Graduate            No   
2  LP001005   Male     Yes          0      Graduate           Yes   
3  LP001006   Male     Yes          0  Not Graduate            No   
4  LP001008   Male      No          0      Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0         NaN             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   
3             2583             2358.0       120.0             360.0   
4             6000                0.0       141.0             360.0   

   Credit_History Property_Ar

In [21]:
df['Loan_Status'] = df['Loan_Status'].map({'Y':1, 'N':0})


In [22]:
# Numerical columns
num_cols = df.select_dtypes(include=['int64','float64']).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Categorical columns
cat_cols = df.select_dtypes(include='object').columns
df[cat_cols] = df[cat_cols].fillna(df[cat_cols].mode().iloc[0])


In [23]:
df = pd.get_dummies(df, drop_first=True)


In [24]:
from sklearn.model_selection import train_test_split

X = df.drop('Loan_Status', axis=1)
y = df['Loan_Status']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
#standardization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

models = {
    "Logistic Regression": Pipeline([
        ("scaler", StandardScaler()),
        ("lr", LogisticRegression(max_iter=1000))
    ]),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "KNN": Pipeline([
        ("scaler", StandardScaler()),
        ("knn", KNeighborsClassifier(n_neighbors=7))
    ]),
    "Random Forest": RandomForestClassifier(
        n_estimators=200,
        random_state=42,
        n_jobs=-1
    )
}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    print(f"{name} Accuracy: {acc:.4f}")


Logistic Regression Accuracy: 0.8455
Decision Tree Accuracy: 0.8049
KNN Accuracy: 0.8455
Random Forest Accuracy: 0.8537


Best Base Model & Why

Best Model: Random Forest

Reason:

Captures non-linear patterns

Handles feature interactions

Robust to noise

Ensemble averaging reduces overfitting

📌 Exam line:

Random Forest performs best due to its ability to model complex relationships and reduce variance.

In [26]:
#Define Base Models
from sklearn.ensemble import StackingClassifier

base_models = [
    ('lr', LogisticRegression(max_iter=1000)),
    ('rf', RandomForestClassifier(n_estimators=150, random_state=42)),
    ('knn', KNeighborsClassifier(n_neighbors=7))
]

In [27]:
#Meta-Model
meta_model = LogisticRegression()

In [28]:
#Stacking Classifier (No Data Leakage)
stack_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5
)

stack_model.fit(X_train, y_train)

StackingClassifier(cv=5,
                   estimators=[('lr', LogisticRegression(max_iter=1000)),
                               ('rf',
                                RandomForestClassifier(n_estimators=150,
                                                       random_state=42)),
                               ('knn', KNeighborsClassifier(n_neighbors=7))],
                   final_estimator=LogisticRegression())

In [29]:
from sklearn.metrics import confusion_matrix
#base Model Evaluation
best_model = models["Random Forest"]
y_pred_base = best_model.predict(X_test)

print("Base Model Accuracy:", accuracy_score(y_test, y_pred_base))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_base))


Base Model Accuracy: 0.8536585365853658
Confusion Matrix:
 [[21 17]
 [ 1 84]]


In [30]:
#Stacking Model Evaluation
y_pred_stack = stack_model.predict(X_test)

print("Stacking Accuracy:", accuracy_score(y_test, y_pred_stack))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_stack))


Stacking Accuracy: 0.8617886178861789
Confusion Matrix:
 [[22 16]
 [ 1 84]]


Stacking improves performance only when base models are diverse and sufficient data is available.

###**DATASET 2: HOUSE PRICE PREDICTION (REGRESSION)**

In [31]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

# Path to the dataset (update if needed)
path = '/content/drive/My Drive/Colab Notebooks/kc_house_data.csv'

# Importing the dataset
df= pd.read_csv(path)
print(df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
           id             date     price  bedrooms  bathrooms  sqft_living  \
0  7129300520  20141013T000000  221900.0         3       1.00         1180   
1  6414100192  20141209T000000  538000.0         3       2.25         2570   
2  5631500400  20150225T000000  180000.0         2       1.00          770   
3  2487200875  20141209T000000  604000.0         4       3.00         1960   
4  1954400510  20150218T000000  510000.0         3       2.00         1680   

   sqft_lot  floors  waterfront  view  ...  grade  sqft_above  sqft_basement  \
0      5650     1.0           0     0  ...      7        1180              0   
1      7242     2.0           0     0  ...      7        2170            400   
2     10000     1.0           0     0  ...      6         770              0   
3      5000     1.0           0     0  ...      7        1050            910   
4 

In [32]:
df = df.dropna()


In [33]:
df.drop('date', axis=1, inplace=True)


In [34]:
X = df.drop('price', axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
#standardisation
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [35]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    )
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"{name} RMSE: {rmse:.2f}")


Linear Regression RMSE: 212520.16
Decision Tree RMSE: 201119.60
Random Forest RMSE: 154144.64


In [36]:
from sklearn.ensemble import StackingRegressor

base_models = [
    ('lr', LinearRegression()),
    ('rf', RandomForestRegressor(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    ))
]

stack_reg = StackingRegressor(
    estimators=base_models,
    final_estimator=LinearRegression(),
    cv=3,
    n_jobs=-1
)

stack_reg.fit(X_train, y_train)


StackingRegressor(cv=3,
                  estimators=[('lr', LinearRegression()),
                              ('rf',
                               RandomForestRegressor(max_depth=10, n_jobs=-1,
                                                     random_state=42))],
                  final_estimator=LinearRegression(), n_jobs=-1)

In [37]:
from sklearn.metrics import r2_score

y_pred_stack = stack_reg.predict(X_test)

rmse_stack = np.sqrt(mean_squared_error(y_test, y_pred_stack))
r2 = r2_score(y_test, y_pred_stack)

print("Stacking RMSE:", rmse_stack)
print("R² Score:", r2)


Stacking RMSE: 153930.98421379476
R² Score: 0.8432645640502553


📌 Dataset 1 — Loan Default (Classification)

Target: Loan_Status (0/1)

Use:

StackingClassifier
LogisticRegression
RandomForestClassifier

📌 Dataset 2 — House Price (Regression)

Target: price (continuous)

Use:

StackingRegressor
LinearRegression
RandomForestRegressor


❗ Never mix classifier with regression target or vice versa.